### Convolutional Neural Network with fully connected layer vs average pooling

In [1]:
import os
import time
import random as rnd
from glob import glob
import numpy as np
import tensorflow as tf
from skimage import color, io
from scipy.misc import imresize

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split

%matplotlib inline
rnd.seed(41)

In [2]:
# Utility methods
def fetch_batch(X, iteration, batch_size):
    i = iteration * batch_size
    j = iteration * batch_size + batch_size
    return X[i:j]

def get_total_tf_parameters():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parametes = 1
        for dim in shape:
            variable_parametes *= dim.value
    
        total_parameters += variable_parametes
    return total_parameters

In [3]:
data_path = './tf-cats-vs-dogs/data/train'

cat_files_path = os.path.join(data_path, 'cat.*.jpg')
dog_files_path = os.path.join(data_path, 'dog.*.jpg')

cat_files = sorted(glob(cat_files_path))
dog_files = sorted(glob(dog_files_path))

file_count = len(cat_files) + len(dog_files)
print('Files found: {}'.format(file_count))

image_size = 128

Files found: 25000


In [4]:
file_count = 11000

allX = np.zeros((file_count, image_size, image_size, 3), dtype='float64')
ally = np.zeros(file_count)
count = 0
for f in cat_files[:5500]:
    try:
        img = io.imread(f)
        new_img = imresize(img, (image_size, image_size, 3))
        new_img = np.array(new_img) / 255.
        allX[count] = new_img
        ally[count] = 0
        count += 1
    except:
        continue

for f in dog_files[:5500]:
    try:
        img = io.imread(f)
        new_img = imresize(img, (image_size, image_size, 3))
        new_img = np.array(new_img) / 255.
        allX[count] = np.array(new_img)
        ally[count] = 1
        count += 1
    except:
        continue
       
# TODO: change to train-validation-test split
# test-train split   
X_train, X_test, Y_train, Y_test = train_test_split(allX, ally, test_size=0.10, random_state=41)
print('X: {} {}'.format(X_train.shape[0], X_train.shape))
print('y: {} {}'.format(Y_train.shape[0], Y_train.shape))   

X: 9900 (9900, 128, 128, 3)
y: 9900 (9900,)


In [5]:
def create_base_model(X):
# all the layers before the final layer   
    # Conv 1
    conv1 = tf.layers.conv2d(
        inputs=X,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pool 1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

    # Conv2 
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # pool 2
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

    # Conv 3
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pool 3
    pool3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

    # Conv4 
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # pool 4
    pool4 = tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

    # Conv 5
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pool 5
    pool5 = tf.nn.max_pool(conv5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

    # Conv6 
    conv6 = tf.layers.conv2d(
        inputs=pool5,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    return conv6

In [6]:
def fc_model(model):
# the fully connected layer before the softmax output layer
    # pool 6
    pool6 = tf.nn.max_pool(model, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    print(pool6.shape)

    pool6_flat = tf.reshape(pool6, shape=[-1, 64 * 2 * 2])

    # Dense Layer
    dense = tf.layers.dense(inputs=pool6_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.5)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    return logits
    

In [7]:
def avg_pool_model(model):
# use avg pooling instead of fully connected layer
    pool6 = tf.nn.avg_pool(model, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    pool6_flat = tf.reshape(pool6, shape=[-1, 64 * 2 * 2])
    
    logits = tf.layers.dense(inputs=pool6_flat, units=2)
    
    return logits

In [8]:
def train(sess, n_epochs=15, batch_size=60):
    init.run()
    print('Training initiated...')
    
    test_accs = []
    
    for epoch in range(n_epochs):
        n_batches = len(X_train) // batch_size
        for i in range(len(X_train) // batch_size):
            X_train_batch = fetch_batch(X_train, i, batch_size)
            Y_train_batch = fetch_batch(Y_train, i, batch_size)
            
            sess.run(training_op, feed_dict={X: X_train_batch, y: Y_train_batch})

        # Test against a random training batch to get an idea of well we are training
        rand = rnd.randint(0, n_batches-1)
        X_batch = fetch_batch(X_train, rand, batch_size)
        Y_batch = fetch_batch(Y_train, rand, batch_size)
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: Y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: Y_test})
        test_accs.append(acc_test)
        
        print(epoch, 'Train acc: {}-{} Test acc: {}'.format(rand, acc_train, acc_test))
    return test_accs
        
        


In [9]:
# Train with fully connected layer
total_fc_acc = 0
total_parameters = 0
total_time = 0
for i in range(10):
    print('Fully connected iteration: {}'.format(i))
    tf.reset_default_graph()
    with tf.Session() as sess:
        X = tf.placeholder(tf.float32, shape=[None, image_size, image_size, 3], name="X")
        y = tf.placeholder(tf.int32, shape=[None], name="y")

        logits = create_base_model(X)
        logits = fc_model(logits)

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
        loss = tf.reduce_mean(xentropy)
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

        # accuracy
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

        init = tf.global_variables_initializer()

        t = time.time()
        test_accs = train(sess)
        total_time += ( time.time() - t)
        total_fc_acc += max(test_accs)

        total_parameters = get_total_tf_parameters()
        
print('\r\nSUMMARY: Fully Connected\r\n==========')
print('Avg validation accuracy: {}'.format(total_fc_acc/5.))
print('Avg training time: {}'.format(total_time/5.))
print('Total parameters: {}\r\n'.format(total_parameters))
    

# Train with avg pooling
total_fc_acc = 0
total_parameters = 0
total_time = 0
for i in range(10):
    print('Avg pooling iteration: {}'.format(i))
    tf.reset_default_graph()      
    with tf.Session() as sess:
        X = tf.placeholder(tf.float32, shape=[None, image_size, image_size, 3], name="X")
        y = tf.placeholder(tf.int32, shape=[None], name="y")

        logits = create_base_model(X)
        logits = avg_pool_model(logits)

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
        loss = tf.reduce_mean(xentropy)
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

        # accuracy
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

        init = tf.global_variables_initializer()

        t = time.time()
        test_accs = train(sess)
        total_time += ( time.time() - t)
        total_fc_acc += max(test_accs)

        total_parameters = get_total_tf_parameters()
        
print('\r\nSUMMARY: Avg Pooling\r\n==========')
print('Avg validation accuracy: {}'.format(total_fc_acc/5.))
print('Avg training time: {}'.format(total_time/5.))
print('Total parameters: {}\r\n'.format(total_parameters))

    


Fully connected iteration: 0
(?, 2, 2, 64)
Training initiated...
0 Train acc: 97-0.5500000715255737 Test acc: 0.613636314868927
1 Train acc: 85-0.6666666865348816 Test acc: 0.6481817960739136
2 Train acc: 59-0.7000000476837158 Test acc: 0.667272686958313
3 Train acc: 42-0.6500000357627869 Test acc: 0.6909090280532837
4 Train acc: 98-0.8333333730697632 Test acc: 0.7063635587692261
5 Train acc: 147-0.7000000476837158 Test acc: 0.7390908002853394
6 Train acc: 72-0.8833334445953369 Test acc: 0.7718181610107422
7 Train acc: 141-0.8000000715255737 Test acc: 0.7809091210365295
8 Train acc: 70-0.9500000476837158 Test acc: 0.7900000214576721
9 Train acc: 98-0.9500000476837158 Test acc: 0.7972727417945862
10 Train acc: 147-0.8666667342185974 Test acc: 0.8027272820472717
11 Train acc: 2-0.8833333849906921 Test acc: 0.7890909314155579
12 Train acc: 63-0.8500000834465027 Test acc: 0.8027273416519165
13 Train acc: 4-0.9000000953674316 Test acc: 0.7899999618530273
14 Train acc: 112-0.9166667461395264

Training initiated...
0 Train acc: 159-0.6833333969116211 Test acc: 0.5618181824684143
1 Train acc: 39-0.6166666746139526 Test acc: 0.6718181371688843
2 Train acc: 126-0.6166666746139526 Test acc: 0.6818180680274963
3 Train acc: 84-0.7000000476837158 Test acc: 0.7436363697052002
4 Train acc: 31-0.6166667342185974 Test acc: 0.7481818199157715
5 Train acc: 58-0.8666666746139526 Test acc: 0.7445454597473145
6 Train acc: 98-0.8166667222976685 Test acc: 0.7827273607254028
7 Train acc: 30-0.8500000238418579 Test acc: 0.7854546308517456
8 Train acc: 114-0.8833333849906921 Test acc: 0.7690908312797546
9 Train acc: 153-0.9166667461395264 Test acc: 0.7900000810623169
10 Train acc: 49-0.9000000953674316 Test acc: 0.7718182802200317
11 Train acc: 141-0.9333333969116211 Test acc: 0.7581816911697388
12 Train acc: 127-0.9166667461395264 Test acc: 0.7618182301521301
13 Train acc: 100-0.9166666865348816 Test acc: 0.7672727108001709
14 Train acc: 91-0.9333333969116211 Test acc: 0.7481818199157715
Fully 

Training initiated...
0 Train acc: 27-0.5666667222976685 Test acc: 0.5790908932685852
1 Train acc: 77-0.6333334445953369 Test acc: 0.589090883731842
2 Train acc: 99-0.5666667222976685 Test acc: 0.5763636231422424
3 Train acc: 17-0.6666667461395264 Test acc: 0.6081817746162415
4 Train acc: 139-0.7333333492279053 Test acc: 0.6499999761581421
5 Train acc: 81-0.6166666746139526 Test acc: 0.6590908765792847
6 Train acc: 112-0.7333333492279053 Test acc: 0.7199999094009399
7 Train acc: 95-0.783333420753479 Test acc: 0.6945454478263855
8 Train acc: 131-0.6833333969116211 Test acc: 0.699999988079071
9 Train acc: 45-0.7333333492279053 Test acc: 0.7245453596115112
10 Train acc: 21-0.7166666984558105 Test acc: 0.7109090089797974
11 Train acc: 18-0.8666666746139526 Test acc: 0.7072726488113403
12 Train acc: 94-0.6833333969116211 Test acc: 0.7145453691482544
13 Train acc: 99-0.7333333492279053 Test acc: 0.7336363196372986
14 Train acc: 89-0.7500000596046448 Test acc: 0.7072726488113403
Avg pooling i